In [1]:

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt


In [2]:
np.random.seed(100)

In [ ]:
# There are going to be 
 
J = 10 

# Across time period 

T = 100 


# This might lead to bias see the assumption about J tending to infty and sthen T statoc
# This is the same sort of panel data bias!! Look a bit into microetrics


In [15]:
# All the parameter vector 
theta = np.array([1., 0.2, 0.7, 0.7, 0.2, 0.5, 0.1, 0.05])
beta_0 = theta[0]
beta_l = theta[1]
beta_k = theta[2]
beta = theta[:3]
# The coeff in the production function stuff 
alpha = theta[3]
sigma_eta = theta[4]
simga_v = theta[5]
simga_w = theta[6]

# This is needed in the capital accumulation equation 
delta = theta[7]

# The parameters for the shock in the labor function 
iota = np.random.normal(0, 1, J)

# The parameter for investment decision and productivity 
gamma = 0.1

# The prices for the companies and the wage
p_eq = np.ones(J)
wage = np.ones(J)/2

# The capital value 
k = np.random.normal(1, 0.5, J)

# The productivity shock has to be drawn from the stationart distribution of an ar(1) process and then it has to
# follow and Ar(1) process to begin with


In [70]:
def production(beta, l, k, prod, eta, J):
    var = np.column_stack((l, k))
    V_0 = np.ones(J)
    inputs = np.column_stack((V_0, var))
    y = inputs@beta + prod + eta
    return y

In [56]:
# Deriving the optimal log_labor function 
# As the variable produciton input labor has to come from some sort of optimality condition in equilibirum
def log_labor_optimal(beta, price, wage, omega, eta, k, iota): 
    beta_0 = beta[0]
    beta_l = beta[1]
    beta_k = beta[2]
    l = (1/(1-beta_l))*(np.log((price*beta_l)/wage)+ beta_0 + omega + iota + eta + beta_k*k)
    return l 

In [75]:
def investment(delta, omega, k):
    # There has to be some structure to the investment decision
    gamma = 0.1 
    inv = (delta + gamma*omega)*np.exp(k)
    return inv

0.40774227426885684
